In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install all needed libraries
!pip install transformers
!pip install datasets
!pip install pypinyin
!pip install pkuseg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.4 MB/s 
     |████████████████████████████████| 120 kB 68.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.8 MB/s 
     |████████████████████████████████| 115 kB 75.2 MB/s 
     |████████████████████████████████| 212 kB 63.9 MB/s 
     |████████████████████████████████| 127 kB 72.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 7.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

In [3]:
# import libraries for transformers, segmentation, pinyin and etc...
from transformers import BertTokenizer, BartForConditionalGeneration, GPT2LMHeadModel, TextGenerationPipeline
import pkuseg
from pypinyin import Style, lazy_pinyin, pinyin
import string
import numpy as np
import torch
import os
import itertools
import random
import math

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
# connect to cuda if available
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# set the tokenizer and model using our fintuned model
model_1 = r'/content/drive/MyDrive/MSc_Project/args/Method 1/checkpoint-9300'
model_21 = r'/content/drive/MyDrive/MSc_Project/args/Method 21/checkpoint-18600'
model_22 = r'/content/drive/MyDrive/MSc_Project/args/Method 22/checkpoint-18900'
model_3 = r'/content/drive/MyDrive/MSc_Project/args/Method 3/checkpoint-70000'

PATH = model_3
tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
model = BartForConditionalGeneration.from_pretrained(os.path.join(PATH, 'pytorch_model.bin'),config=os.path.join(PATH, 'config.json'), local_files_only=True)

# move our model to the selected device
model.to(device)

seg = pkuseg.pkuseg()

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [5]:
if PATH == model_22:
  # adding special mask tokens for each rhyming group 
  num_added_toks = tokenizer.add_tokens('[A]')
  num_added_toks = tokenizer.add_tokens(['[B]'])
  num_added_toks = tokenizer.add_tokens(['[C]'])
  num_added_toks = tokenizer.add_tokens(['[D]'])
  num_added_toks = tokenizer.add_tokens(['[E]'])
  num_added_toks = tokenizer.add_tokens(['[F]'])
  num_added_toks = tokenizer.add_tokens(['[G]'])
  num_added_toks = tokenizer.add_tokens(['[H]'])
  num_added_toks = tokenizer.add_tokens(['[I]'])
  num_added_toks = tokenizer.add_tokens(['[J]'])
  num_added_toks = tokenizer.add_tokens(['[K]'])
  num_added_toks = tokenizer.add_tokens(['[L]'])
  num_added_toks = tokenizer.add_tokens(['[M]'])
  model.resize_token_embeddings(len(tokenizer)) 


if PATH == model_3:
  num_added_toks = tokenizer.add_tokens(['[RHYME]'])
  model.resize_token_embeddings(len(tokenizer)) 

In [6]:
train_schema_name = []
with open('/content/drive/MyDrive/MSc_Project/dataset/train_schema_name.txt', 'r', encoding='utf-8') as ep:
    for line in ep:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        train_schema_name.append(x)

train_schema_prob = []
with open('/content/drive/MyDrive/MSc_Project/dataset/train_schema_prob.txt', 'r', encoding='utf-8') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = int(line[:-1])

        # add current item to the list
        train_schema_prob.append(x)

test_schema_name = []
with open('/content/drive/MyDrive/MSc_Project/dataset/test_schema_name.txt', 'r', encoding='utf-8') as ap:
    for line in ap:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        test_schema_name.append(x)

test_schema_prob = []
with open('/content/drive/MyDrive/MSc_Project/dataset/test_schema_prob.txt', 'r', encoding='utf-8') as bp:
    for line in bp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = int(line[:-1])

        # add current item to the list
        test_schema_prob.append(x)

UER_schema_name = []
with open('/content/drive/MyDrive/MSc_Project/dataset/UER_schema_name.txt', 'r', encoding='utf-8') as cp:
    for line in cp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        UER_schema_name.append(x)

UER_schema_prob = []
with open('/content/drive/MyDrive/MSc_Project/dataset/UER_schema_prob.txt', 'r', encoding='utf-8') as dp:
    for line in dp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = int(line[:-1])

        # add current item to the list
        UER_schema_prob.append(x)

def schema_produce(schema, schema_name, schema_prob, num = 5):
    list_a = [schema]
    name_new = schema_name[:]
    prob_new = schema_prob[:]
    if schema in schema_name:
        ind = name_new.index(schema)
        name_new.remove(schema)
        prob_new = schema_prob[:ind] + schema_prob[ind+1:]
    list_a+=random.choices(name_new, weights=prob_new, cum_weights=None, k=num-1)
    return list_a

def schema_produce1(schema, schema_name, schema_prob, num = 5):
    list_a = random.choices(schema_name, weights=schema_prob, cum_weights=None, k=num)
    return list_a

In [7]:
test_data = []
with open(r'/content/drive/MyDrive/MSc_Project/dataset/test_data_10_new.txt', 'r') as ttd:
    for line in ttd:
        line = line.rstrip()
        test_data.append(line)

UER_data = []
with open(r'/content/drive/MyDrive/MSc_Project/dataset/UER_test_data_generation.txt', 'r') as td:
    for line in td:
        line = line.rstrip()
        UER_data.append(line)

In [8]:
# Get the alphabet of capital letters
alphabet = string.ascii_letters[26:]

# The punctuation list
PUNCS = set([",", ".", "?", "!", ":", "，", "。", "？", "！", "："])

# The yunmu(final) dictionary for rhyming: all yunmus(finals) in the same number group rhyme.
yunmus= {
            "0":["a", "ia", "ua", "va", "üa"],
            "1":["e", "o", "uo", "ie", "ue", "üe", "ve"],
            "2":["u"],
            "3":["i", "ü", "v"],
            "4":["ai", "uai"],
            "5":["ao", "iao"],
            "6":["ou", "iu", "iou"],
            "7":["an", "ian", "uan", "üan", "van"],
            "8":["en", "in", "un", "ün", "vn"],
            "9":["ang", "iang", "uang"],
            "10":["eng", "ing", "ueng", "ong", "iong"],
            "11":["er"],
            "12":["ei", "ui", "uei", "vei"],
           }

yun2id = {}
for yid, yws in yunmus.items():
    for w in yws:
        yun2id[w] = yid


In [9]:
# Return the yunmu of the last word in the input text
def yunmu_name(text):
    w = text[-1]
    if w in PUNCS and len(text) > 1:
        w = text[-2]
    yunmu = lazy_pinyin(w, style=Style.FINALS)
    yunmu = yunmu[0]
    return yunmu

# Return the corresponding yunmu_id of the yunmu
def yunmu_id(yunmu):
    if yunmu in yun2id:
        id = yun2id[yunmu]
    else:
        id = '-1'
    return id

# Return the corresponding schema of the sentences
def sents_to_schema(sents):
    m = 0
    all_id = []
    schema = ''
    for sent in sents:
        y_id = int(yunmu_id(yunmu_name(sent)))
        if y_id in all_id:
            schema += schema[all_id.index(y_id)]
        else:
            schema += alphabet[m]
            m += 1
        all_id.append(y_id)
    return schema

# clean the inputs, get rid of the spaces and punctuations and put the text into sentences in a list
def clean_text(text):
    text = text.replace(" ", "")
    for punc in PUNCS:
        text = text.replace(punc, " ")
    sents = text.split()
    return sents

def rhyme_list(schema):
    rhy_map = {}
    for i, r in enumerate(schema):
        if r in rhy_map:
            rhy_map[r] += [i]
        else:
            rhy_map[r] = [i]
    rhy_list = []
    for r in rhy_map:
        rhy_list.append(rhy_map[r])
    return rhy_list

In [10]:
def encode_text_1(sents, schema):
    new_sents = []
    for i in range(len(sents)):
        ori_text = sents[i]
        mask_word = seg.cut(ori_text)[-1]
        new_sents.append(ori_text[:len(ori_text)-len(mask_word)] + '[MASK]')
    new_text = '，'.join(new_sents)
    return new_text
    
def encode_text_21(sents, schema):
    new_sents = []
    for i in range(len(sents)):
        ori_text = sents[i]
        mask_word = seg.cut(ori_text)[-1]
        new_sents.append(ori_text[:len(ori_text)-len(mask_word)] + '[MASK]')
    new_text = '，'.join(new_sents)
    return new_text + '[SEP]' + schema

def encode_text_22(sents, schema):
    new_sents = []
    for i in range(len(sents)):
        ori_text = sents[i]
        mask_word = seg.cut(ori_text)[-1]
        new_sents.append(ori_text[:len(ori_text)-len(mask_word)] + '[' + schema[i] + ']')
    new_text = '，'.join(new_sents)
    return new_text

def encode_text_3(sents, rhy_list):
    new_sents = list(sents)
    for i in rhy_list:
        ori_text = sents[i]
        segmentation = seg.cut(ori_text)
        mask_word = segmentation[-1]
        new_sents[i] = ori_text[:len(ori_text)-len(mask_word)] + '[RHYME]'
    new_text = '，'.join(new_sents)
    return new_text


In [11]:
# Evaluation model for perplexity
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
eval_tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-lyric")
eval_model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-lyric")

def eval_ppl(text):
    encodings = eval_tokenizer(text, return_tensors="pt")
    max_length = eval_model.config.n_positions
    length = encodings.input_ids.size(1)

    trg_len = length
    input_ids = encodings.input_ids[:, length - max_length:]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = eval_model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    ppl = torch.exp(neg_log_likelihood / length)
    return ppl

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

In [12]:
def replacement(text, need_id):
    #tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
    #model = BartForConditionalGeneration.from_pretrained("fnlp/bart-base-chinese")
    #seg = pkuseg.pkuseg()  

    mask_word = seg.cut(text)[-1]
    remain_text = text.replace(mask_word, "")
    new_text = text.replace(mask_word, "[MASK]")
    batch = tokenizer(new_text, return_tensors="pt")
    batch.to(device)
    generated_ids = model.generate(batch["input_ids"], num_return_sequences = 100, max_length = 20, do_sample = True, temperature = 2.0, output_scores = True, return_dict_in_generate=True)
    result = tokenizer.batch_decode(generated_ids["sequences"], skip_special_tokens=True)
    result = [s.replace(" ", "") for s in result]
    result = list(set(result))
    result = [s for s in result if len(s) > 0]
    if len(result) == 0:
      return text
    #scores = [float(s) for s in generated_ids['sequences_scores']]

    '''
    if need_count:
        return replacement_count(result)

    if need_check:
        result = replacement_check(result, remain_text)
    '''
    return sent_select(result, need_id)

def sent_select(result, need_id):
    yun_id = [yunmu_id(yunmu_name(s)) for s in result]
    ppl = math.inf
    ans = result[0]
    for i in range(len(result)):
        if yun_id[i] == need_id:
            new_ppl = eval_ppl(result[i])
            if new_ppl < ppl:
                ppl = new_ppl
                ans = result[i]
    return ans

def replacement_check(result, remain_text):
    new_result = []
    length = len(remain_text)
    for i in range(len(result)):
        test = result[i][:length]
        if remain_text == test:
            new_result.append(result[i])
    if len(new_result) == 0:
        return result
    return new_result

def replacement_count(result):
    gen_count = {}
    for i in result:
        gen_count[i] = result.count(i)
    return gen_count

In [13]:

def scheme_mapping(rhyme_scheme):
    rhyme_scheme = rhyme_scheme.upper()
    scheme_num = list(rhyme_scheme)
    for i in range(len(scheme_num)):
        scheme_num[i] = alphabet.index(scheme_num[i])
    scheme_map = {}
    for i, r in enumerate(scheme_num):
        if r in scheme_map:
            scheme_map[r] += [i]
        else:
            scheme_map[r] = [i]
    return scheme_map

def rhyme_pairs(rhyme_scheme):
    map = scheme_mapping(rhyme_scheme)
    pairs = []
    for j in map:
        list_rhyme = map[j]
        for i in range(len(list_rhyme)-1):
            pairs += list(zip(list_rhyme, list_rhyme[i+1:]))
    return pairs

def non_rhyme_pairs(rhyme_scheme):
    map = scheme_mapping(rhyme_scheme)
    list_all = [i for i in range(len(rhyme_scheme))]
    non_pairs = []
    for j in map:
        list_rhyme = map[j]
        list_all = list(set(list_all) - set(list_rhyme))
        all = list(itertools.product(list_rhyme, list_all))
        non_pairs += [(k,i) if i>k else (i,k) for (i,k) in all]
    return non_pairs

def all_metric(old_rhyme, new_rhyme):
    rhyme_old = rhyme_pairs(old_rhyme)
    nonrhyme_old = non_rhyme_pairs(old_rhyme)
    rhyme_new = rhyme_pairs(new_rhyme)
    TP = len(list(set(rhyme_old) & set(rhyme_new)))
    FP = len(list(set(nonrhyme_old) & set(rhyme_new)))
    precision = TP/len(rhyme_new)
    recall = TP/len(rhyme_old)
    false_pos = FP/len(nonrhyme_old)
    score = TP/(FP+len(rhyme_old))
    return [precision, recall, false_pos, score]

def list_all_metric(old_rhyme, result, length = 10):
    pre_fp = [0, 0, 0, -100]
    answer = ''
    for i in result:
      new_sents = clean_text(i)[:length]
      if len(new_sents) == 10:
        new_rhyme = sents_to_schema(new_sents)
        if new_rhyme != 'AAAAAAAAAA' and new_rhyme != 'ABCDEFGHIJ':
          new_pre_fp = all_metric(old_rhyme, new_rhyme)
          if new_pre_fp[-1] > pre_fp[-1]:
              pre_fp = new_pre_fp
              answer = '，'.join(new_sents)
    ppl = float(eval_ppl(answer))
    pre_fp.append(ppl)
    return pre_fp, answer

In [14]:
def single_test(text, schema_name, schema_prob, modelid = 1, length = 10):
  if modelid == 1:
    encode_text = encode_text_1
  elif modelid == 21:
    encode_text = encode_text_21
  elif modelid == 22:
    encode_text = encode_text_22
  sents = clean_text(text)[:length]
  ori_schema = sents_to_schema(sents)
  schema_list = schema_produce(ori_schema, schema_name, schema_prob, num = 3)
  final_pre_fp_ppl = [0,0,0,0,0]
  final_generation = []
  for schema in schema_list:
    new_text = encode_text(sents[:length], schema)
    batch = tokenizer(new_text, return_tensors="pt")
    batch.to(device)
    generated_ids = model.generate(batch["input_ids"], num_return_sequences = 50, max_length = 128, do_sample = True, temperature = 1.0, output_scores = True, return_dict_in_generate=True)
    result = tokenizer.batch_decode(generated_ids["sequences"], skip_special_tokens=True)
    result = [s.replace(" ", "") for s in result]
    best_pre_fp_ppl, best_generation = list_all_metric(schema, result, length)
    final_pre_fp_ppl = [sum(value) for value in zip(final_pre_fp_ppl, best_pre_fp_ppl)]
    final_generation.append(best_generation)
  return [round(x / len(schema_list), 3) for x in final_pre_fp_ppl], final_generation


In [15]:
def single_test3(text, schema_name, schema_prob, length = 10):
  sents = clean_text(text)
  ori_schema = sents_to_schema(sents)
  schema_list = schema_produce(ori_schema, schema_name, schema_prob, num = 3)
  final_pre_fp_ppl = [0,0,0,0,0]
  final_generation = []
  for schema in schema_list:
    rhy_list = rhyme_list(schema)
    ori_sents = sents[:length]
    for j in range(len(rhy_list)):
        new_text = encode_text_3(ori_sents, rhy_list[j])
        batch = tokenizer(new_text, return_tensors="pt")
        batch.to(device)
        generated_ids = model.generate(batch["input_ids"], num_return_sequences = 50, max_length = 128, num_beams = 50, do_sample = False, temperature = 1.0, output_scores = True, return_dict_in_generate=True)
        result = tokenizer.batch_decode(generated_ids["sequences"], skip_special_tokens=True)
        result = [s.replace(" ", "") for s in result]
        best_pre_fp_ppl, best_generation = list_all_metric(schema, result, length)
        if best_generation == '':
          best_generation = text
          best_pre_fp_ppl = all_metric(schema, ori_schema)
          ppl = float(eval_ppl(text))
          best_pre_fp_ppl.append(ppl)
        ori_sents = clean_text(best_generation)
    final_pre_fp_ppl = [sum(value) for value in zip(final_pre_fp_ppl, best_pre_fp_ppl)]
    final_generation.append(best_generation)
  return [round(x / len(schema_list), 3) for x in final_pre_fp_ppl], final_generation


In [16]:
def single_test111(text, schema_name, schema_prob, length = 10):
  sents = clean_text(text)[:length]
  ori_schema = sents_to_schema(sents)
  schema_list = schema_produce(ori_schema, schema_name, schema_prob, num = 3)
  final_pre_fp_ppl = [0,0,0,0,0]
  final_generation = []
  for schema in schema_list:
    # Reconstruct rhyme_scheme into scheme_map
    scheme_map = scheme_mapping(schema)
    new_sents = sents[:]
    for i in range(len(scheme_map)):
        # if the sent doesn't need to rhyme, skip it
        if len(scheme_map[i]) == 1:
            pass
        else:
            # Save the first sentence in the same rhyme group
            mapping = scheme_map[i][1:]
            sample_num = scheme_map[i][0]
            sample = sents[sample_num]
            need_id = yunmu_id(yunmu_name(sample))
            
            # Regenerate other sentences according to the rhyme of the first one
            for j in mapping:
                text = sents[j]
                if yunmu_id(yunmu_name(text)) != need_id:
                    generated_text = replacement(text, need_id)
                    new_sents[j] = generated_text
    
    # Produce the result text
    post_text = ','.join(new_sents)

    # Produce the metrics
    new_rhyme = sents_to_schema(new_sents)
    new_pre_fp = all_metric(schema, new_rhyme)
    ppl = float(eval_ppl(post_text))
    new_pre_fp.append(ppl)

    final_pre_fp_ppl = [sum(value) for value in zip(final_pre_fp_ppl, new_pre_fp)]
    final_generation.append(post_text)
  return [round(x / len(schema_list), 3) for x in final_pre_fp_ppl], final_generation


In [17]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
ppl_all = []
for i in test_data:
  ppl = float(eval_ppl(i))
  ppl_all.append(ppl)


In [ ]:
len(ppl_all)

870

In [ ]:
np.savetxt(r'/content/drive/MyDrive/MSc_Project/test_ppl.csv', 
           ppl_all,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
static_all = []
generation_all = []
for i in UER_data[400:]:
  static_1, generation_1 = single_test3(i, train_schema_name, train_schema_prob, length = 10)
  static_all.append(static_1)
  generation_all.append(generation_1)


In [ ]:
np.savetxt(r'/content/drive/MyDrive/MSc_Project/all_UER_model_3.csv', 
           static_all,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
static_all = []
generation_all = []
for i in UER_data[215:400]:
  static_1, generation_1 = single_test3(i, train_schema_name, train_schema_prob, length = 10)
  static_all.append(static_1)
  generation_all.append(generation_1)


In [ ]:
import numpy as np

# using the savetxt 
# from the numpy module
np.savetxt(r'/content/drive/MyDrive/MSc_Project/400_UER_model_3.csv', 
           static_all,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
def all_metric(old_rhyme, new_rhyme):
    rhyme_old = rhyme_pairs(old_rhyme)
    nonrhyme_old = non_rhyme_pairs(old_rhyme)
    rhyme_new = rhyme_pairs(new_rhyme)
    nonrhyme_new = non_rhyme_pairs(new_rhyme)
    TP = len(list(set(rhyme_old) & set(rhyme_new)))
    TN = len(list(set(nonrhyme_old) & set(nonrhyme_new)))
    FP = len(list(set(nonrhyme_old) & set(rhyme_new)))
    FN = len(list(set(rhyme_old) & set(nonrhyme_new)))
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    false_pos = FP/(FP+TN)
    score = TP/(TP+FP+FN)
    return [precision, recall, false_pos, score]